In [ ]:
import os

os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"

import bilby
import corner
import h5py
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import glob
import numpy as np
import pandas as pd

from importlib import reload
import thesis_utils
from thesis_utils.gw import get_cbc_parameter_labels

reload(thesis_utils.gw.utils)

from thesis_utils.plotting import (
    get_default_corner_kwargs,
    set_plotting,
    get_default_figsize,
    save_figure,
)


set_plotting()

In [ ]:
results = dict(
    nessai=bilby.core.result.read_in_result(
        "/home/michael.williams/git_repos/nessai-gwtc-1/analysis/IMRPhenomPv2/outdir_nessai_gwtc_1_GW170823_cal_reweight_Pv2/final_result/nessai_gwtc_1_GW170823_Pv2_data0_1187529256-517903_analysis_H1L1_merge_result.hdf5"
    ),
    inessai_cal=bilby.core.result.read_in_result(
        "/home/michael.williams/git_repos/nessai-gwtc-1/analysis/inessai/outdir_inessai_gwtc_1_GW170823_cal_sample_Pv2_v2/final_result/inessai_gwtc_1_GW170823_Pv2_data0_1187529256-517903_analysis_H1L1_merge_result.hdf5"
    ),
    inessai=bilby.core.result.read_in_result(
        "/home/michael.williams/git_repos/nessai-gwtc-1/analysis/inessai/outdir_inessai_gwtc_1_GW170823_cal_reweight_Pv2_v3/final_result/inessai_gwtc_1_GW170823_Pv2_data0_1187529256-517903_analysis_H1L1_merge_result.hdf5"
    ),
)

In [ ]:
results["inessai"].posterior.keys()

In [ ]:
for key, result in results.items():
    print(key)
    print(f"N post: {len(result.posterior)}")
    print(f"ln-evidence: {result.log_evidence} +/- {result.log_evidence_err}")
    print(f"N like: {result.num_likelihood_evaluations}")
    print(f"Sampling time: {result.sampling_time}")
    print("\n")

In [ ]:
result.outdir

In [ ]:
nessai_results = {}
for key, result in results.items():
    nessai_results[key] = {
        "sampling_time": [],
        "likelihood_evaluations": [],
        "likelihood_time": [],
    }
    rfs = glob.glob(result.outdir + "/*_nessai/result.hdf5")
    for rf in rfs:
        with h5py.File(rf, "r") as f:
            nessai_results[key]["sampling_time"].append(f["sampling_time"][()] / 60)
            nessai_results[key]["likelihood_evaluations"].append(
                f["total_likelihood_evaluations"][()]
            )

In [ ]:
for key, res in nessai_results.items():
    print(key)
    for stat, val in res.items():
        print(stat, np.mean(val))

In [ ]:
ess = {
    "nessai": [6770.5, 6748.2],
    "inessai": [23043.7, 7566.6],
    "inessai_cal": [7290.6, 3347.3],
}
for key, vals in ess.items():
    print(key, np.mean(vals))

In [ ]:
keys = ["num_likelihood_evaluations", "sampling_time"]
table_data = {}
for name, result in results.items():
    table_data[name] = {}
    for key in keys:
        table_data[name][key] = getattr(result, key)

In [ ]:
table_df = pd.DataFrame.from_dict(table_data, orient="index")

In [ ]:
table_df

In [ ]:
parameters = ["mass_1", "mass_2", "chi_eff", "chi_p", "theta_jn", "ra", "dec"]

In [ ]:
figsize = 1.6 * get_default_figsize()
figsize[0] = figsize[1]
fig = plt.figure(figsize=figsize)
colours = {
    "nessai": "k",
    "inessai": "C1",
    "inessai_cal": "C0",
}
labels = get_cbc_parameter_labels(parameters, units=True)

corner_kwargs = get_default_corner_kwargs()
corner_kwargs.pop("color")
corner_kwargs["fill_contours"] = False
corner_kwargs["show_titles"] = False
corner_kwargs["quantiles"] = None
corner_kwargs["plot_datapoints"] = False
corner_kwargs["plot_density"] = False

for key, result in results.items():
    corner_kwargs["hist_kwargs"]["color"] = colours.get(key)
    fig = corner.corner(
        result.posterior[parameters].to_numpy(),
        fig=fig,
        color=colours.get(key),
        labels=labels,
        labelpad=0.1,
        **corner_kwargs,
    )

legend_handles = [
    Line2D([0], [0], label=r"\texttt{nessai}", color="k"),
    Line2D([0], [0], label=r"\texttt{i-nessai} - Calibration sampling", color="C0"),
    Line2D([0], [0], label=r"\texttt{i-nessai} - Calibration reweighting", color="C1"),
]
fig.legend(handles=legend_handles, loc="upper right", fontsize=12)

plt.show()
save_figure(fig, "GW170823_posterior", "figures/inessai/")

In [ ]:
from pesummary.utils.utils import jensen_shannon_divergence
import itertools

In [ ]:
js_parameters = results["inessai"].search_parameter_keys

In [ ]:
js = {}
pairs = [
    ("nessai", "inessai"),
    ("nessai", "inessai_cal"),
    ("inessai", "inessai_cal"),
]
for s1, s2 in pairs:
    label = f"{s1}-{s2}"
    js[label] = {}
    for key in js_parameters:
        js[label][key] = 1000 * jensen_shannon_divergence(
            [results[s1].posterior[key], results[s2].posterior[key]],
            base=2,
        )

In [ ]:
labels = {
    "nessai-inessai": r"\texttt{nessai}-\texttt{i-nessai}",
    "nessai-inessai_cal": r"\texttt{nessai}-\texttt{i-nessai} w. cal",
    "inessai-inessai_cal": r"\texttt{i-nessai}-\texttt{i-nessai} w. cal",
}

In [ ]:
js_df = pd.DataFrame.from_dict(js, orient="index")
js_df = js_df.rename(labels)
js_df = js_df.rename({p: get_cbc_parameter_labels(p) for p in js_parameters}, axis=1)

In [ ]:
get_cbc_parameter_labels("zenith")

In [ ]:
js_df

In [ ]:
tex_table = js_df.style.format(precision=2).to_latex()
with open("results/GW170823_inessai_table.tex", "w") as f:
    f.write(tex_table)

In [ ]:
tex_table

In [ ]:
len(results["inessai_cal"].posterior)